In [1]:
import requests
import pandas as pd
import os
from bs4 import BeautifulSoup

url = "https://time-in.ru/coordinates/Russia"
url_edit = ""
countries = ['germany', 'denmark', 'greece', 'switzerland', 'spain', 'austria']
countries_on_russian = ['Германия', 'Дания', 'Греция', 'Швейцария', 'Испания', 'Австрия']
countries_for_datafarme = []
all_cities = []
latitude = []
longitude = []
result_data = pd.DataFrame()

# Переменная для создания списка с русскими названиями стран в датафрейме
for_dataframe = 0

# Цикл по странам
for country in countries:
    counter = 0
    
    # Меняем ссылку для поиска городов по необходимой стране
    url_edit = url.replace("Russia", country)
    r = requests.get(url_edit)
    soup = BeautifulSoup(r.text, 'html.parser')
    
    # Цикл по городам на сайте
    for i in soup.find_all('li'):
        
        # Пропускаем первые 11 строк с ненужной информацией
        if counter >= 11:

            # Подставляем название страны на русском
            countries_for_datafarme.append(countries_on_russian[for_dataframe])
            
            # Находим <a> с нужным атрибутом class и извлекаем из него текст
            city = i.find('a', {'class':'coordinates-items-left'}).text
            all_cities.append(city)
            
            # Находим <div> с нужным атрибутом class и извлекаем из него текст
            coordinates = i.find('div', {'class': 'coordinates-items-right'}).text

            # Разделяем координаты нга широту и долготу
            latitude_value, longitude_value = map(float, coordinates.split(','))

            # Добавляем коордианты в отдельные списки
            latitude.append(latitude_value)
            longitude.append(longitude_value)

        else: pass
        counter += 1
        
    for_dataframe += 1

# Формируем датафрейм из городов и их координат
result_data['name'] = all_cities
result_data['country'] = countries_for_datafarme
result_data['latitude'] = latitude
result_data['longitude'] = longitude


# Создаем json файл и сохраняем в нужную папку
result_data_json = result_data.to_json('../data/source/cities_data.json', orient='records')

print("Данные были сохранены в файл 'cities_data.json', папка data/source")

Данные были сохранены в файл 'cities_data.json', папка data/source
